# Pandas Performance

In this notebook we will be exploring the performance differences between different approaches of iterating through a Pandas column.  This is based on a post (that apparently no longer exists...): https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6

First we will start by loading our data.  The data is from Lyft's Go Bike program and inclues every trip from 2017: https://www.lyft.com/bikes/bay-wheels/system-data

In [ ]:
import pandas as pd

#change this to local path if not running on ROSIE
df = pd.read_csv('/data/cs2300/examples/2017-fordgobike-tripdata.csv', 
                 dtype={"start_station_latitude":float, "start_station_longitude":float,
                       "end_station_latitude":float, "end_station_longitude":float})
df.head()

Next we define a function to calculate distance based on two GPS locations

In [ ]:
import numpy as np

# Define a basic Haversine distance formula
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

The slowest approach is to loop through the dataframe using `iloc`

In [ ]:
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(df['start_station_latitude'].iloc[i], df['start_station_longitude'].iloc[i], 
                      df['end_station_latitude'].iloc[i], df['end_station_longitude'].iloc[i])
        distance_list.append(d)
    return distance_list
%time df['distance'] = haversine_looping(df)

In [ ]:
#how many rows
print(df.shape)
df.head()

Next, lets try using `iterrows()`

In [ ]:
%%time
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(row['start_station_latitude'], row['start_station_longitude'], 
                                      row['end_station_latitude'], row['end_station_longitude']))
df['distance'] = haversine_series

Next, lets use some functional programming!  Try using `apply`

In [ ]:
%time df['distance'] = df.apply(lambda row: haversine(row['start_station_latitude'], \
                                                      row['start_station_longitude'], \
                                                      row['end_station_latitude'], \
                                                      row['end_station_longitude']), axis=1)

Lets vectorize!

In [ ]:
%time df['distance'] = haversine(df['start_station_latitude'], df['start_station_longitude'], \
                                 df['end_station_latitude'], df['end_station_longitude'])

Lets try numpy vectorize

In [ ]:
%time df['distance'] = haversine(df['start_station_latitude'], df['start_station_longitude'], \
                                 df['end_station_latitude'].values, df['end_station_longitude'].values)

Use matplotlib to make a bar chart summarizing the performance data. Feel free to hard-code in the runtime values found above.